<a href="https://colab.research.google.com/github/kuberiitb/retail_price_ai_agent/blob/main/RetailPriceAgent_Text2Sql_V4withSupervisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agent created using code from [LangGraph Tutorial](https://python.langchain.com/docs/tutorials/sql_qa/)

In [1]:
!pip install langchain langchain_community langchain-openai langgraph ipython --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


# Setting up SQL data

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///retail_price_agent.db")
print(db.get_usable_table_names())

['competitior_information', 'current_product_information', 'forecast_data', 'historical_data', 'inventory_data']


In [3]:
for table_name in db.get_usable_table_names():
  print(table_name)
  print(db.get_table_info([table_name]))
  print("="*80)

competitior_information

CREATE TABLE competitior_information (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	unit_price REAL, 
	promotion TEXT, 
	discount_pct REAL
)

/*
3 rows from competitior_information table:
sku_id	product_name	category	unit_price	promotion	discount_pct
1	t-shirt	men	864.5	BTGO	0.3
8	t-shirt	women	946.4	BTGO	0.3
*/
current_product_information

CREATE TABLE current_product_information (
	sku_id INTEGER, 
	base_price REAL, 
	base_demand REAL, 
	elasticity REAL, 
	margin REAL
)

/*
3 rows from current_product_information table:
sku_id	base_price	base_demand	elasticity	margin
1	1460.0	500.0	-1.06	0.94
8	1820.0	800.0	-0.93	1.08
*/
forecast_data

CREATE TABLE forecast_data (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	date TIMESTAMP, 
	units_sale REAL
)

/*
3 rows from forecast_data table:
sku_id	product_name	category	date	units_sale
1	t-shirt	men	2025-09-01 00:00:00	41.0
1	t-shirt	men	2025-10-01 00:00:00	41.0
1	t-shirt	men	2025-11-01 00:00:00	4

In [4]:
# Langgraph State
from typing_extensions import TypedDict

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

# Setup LLM

In [5]:
import os
from dotenv import load_dotenv
load_dotenv(".env")

True

In [6]:
# from langchain.chat_models import ChatOpenAI

if not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [7]:
llm.invoke("Hi").content

'Hello! How can I assist you today?'

# Using inbuilt Langgraph ReAct agent

In [8]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for t in tools:
  print(t.name, ":", t.description)

sql_db_query : Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
sql_db_schema : Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
sql_db_list_tables : Input is an empty string, output is a comma-separated list of tables in the database.
sql_db_query_checker : Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!


In [9]:
database_information = """"
TABLE: historical_data - Contains historical monthly-level sales, pricing, and profit data for each SKU.

COLUMN: sku_id - Unique identifier for each product-category combination.

COLUMN: product_name - Name of the specific product.

COLUMN: category - Category or department the product belongs to.

COLUMN: date - Month (or month-start date) representing the sales period.

COLUMN: unit_price - Actual selling price per unit after applying discounts.

COLUMN: unit_cost - Cost to produce or acquire one unit of the product.

COLUMN: discount_pct - Percentage discount applied to the base price.

COLUMN: seasonality_factor - Seasonal adjustment factor reflecting demand fluctuations.

COLUMN: units_sold - Total quantity sold for the SKU during the given period.

COLUMN: revenue - Total revenue generated = unit_price * units_sold.

COLUMN: profit - Total profit = (unit_price - unit_cost) * units_sold.

#########################################################################################

TABLE: current_product_information - Stores reference or metadata information about each of our SKU used for pricing and forecasting.

COLUMN: sku_id - Unique identifier for each product-category combination.

COLUMN: base_price - Reference or standard list price of the product.

COLUMN: base_demand - Baseline expected demand level for the product.

COLUMN: elasticity - Price elasticity coefficient indicating sensitivity of demand to price changes.

COLUMN: margin - Target profit margin ratio derived from elasticity or business rules.

#########################################################################################
TABLE: forecast_data - Contains forecasted monthly unit sales for each SKU based on predictive modeling.

COLUMN: sku_id - Unique identifier for each product-category combination.

COLUMN: product_name - Name of the specific product.

COLUMN: category - Category or department the product belongs to.

COLUMN: date - Forecast month or future period.

COLUMN: units_sale - Forecasted number of units expected to be sold.

#########################################################################################

TABLE: inventory_data - Tracks current stock levels for each SKU in the inventory.

COLUMN: sku_id - Unique identifier for each product-category combination.

COLUMN: product_name - Name of the specific product.

COLUMN: category - Category or department the product belongs to.

COLUMN: stock - Current quantity of the SKU available in inventory.

#########################################################################################

TABLE: competitior_information - Captures competitor pricing and promotion details for comparative analysis.

COLUMN: sku_id - Unique identifier representing the same or equivalent SKU.

COLUMN: product_name - Name of the product for cross-reference with competitors.

COLUMN: category - Product category for comparison.

COLUMN: unit_price - Competitor’s selling price for the product.

COLUMN: promotion - Competitor’s promotion or offer label (e.g., "BOGO","NONE" or discount value).

COLUMN: discount_pct - Discount percentage applied by the competitor, if available.
"""

In [11]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain.memory import ConversationBufferMemory

extra_details_about_data = """
1. SKU means product_name+category combination, can be represented by sku_id
2. product means product_name unless specified otherwise.
3. While returning SKU information, mention its details like product_name and category.
4. All the price and revenue information is in INR.
"""

system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

{extra_details_about_data}

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Use this description of tables and columns for reference.
{database_information}

Then you should query the schema of the most relevant tables.

Business context:
Product usually means product-category combination.
""".format(
    dialect="SQLite",
    top_k=5,
    database_information=database_information,
    extra_details_about_data=extra_details_about_data
)

data_agent = create_react_agent(
    name="Retail_Data_Agent",
    model=llm,
    tools=tools,
    prompt=system_message,
)

/tmp/ipython-input-3107907960.py:48: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  data_agent = create_react_agent(


In [12]:
question = "Which is highest selling SKU?"
data_agent.invoke({"messages": [{"role": "user", "content": question}]})

{'messages': [HumanMessage(content='Which is highest selling SKU?', additional_kwargs={}, response_metadata={}, id='0e2c2054-b72e-4af6-abcb-e155d847757c'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fK4PuB2uGZticwjqvNuhLNRQ', 'function': {'arguments': '{"table_names":"historical_data"}', 'name': 'sql_db_schema'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 1139, 'total_tokens': 1157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CRsHFEuXg24A4CcezRHVia7PNWPvL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='Retail_Data_Agent', id='run--e2232681-09f2-4b88-a74b-7998c46d5aa3-0', tool_calls=[{'name': 'sql_

## Example with pretty printing

In [ ]:
question = "Which is highest selling SKU_ID?"

for step in data_agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which is highest selling SKU_ID?
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_list_tables (call_dnY3Mjv4NHrWlmQBSYCFh1ex)
 Call ID: call_dnY3Mjv4NHrWlmQBSYCFh1ex
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

competitior_information, current_product_information, forecast_data, historical_data, inventory_data
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_schema (call_XqAkkawm2pft9vGKrW7EiU9m)
 Call ID: call_XqAkkawm2pft9vGKrW7EiU9m
  Args:
    table_names: historical_data
  sql_db_schema (call_NVbwMqXBP2uH5DXTeXrs5Lnq)
 Call ID: call_NVbwMqXBP2uH5DXTeXrs5Lnq
  Args:
    table_names: current_product_information
================================= Tool Message =========================

## Returning final output only

## skipping all the intermediate steps

In [13]:
from IPython.display import display, Markdown

In [14]:
def get_sql_reponse(question, verbose=False):
  for step in data_agent.stream(
      {"messages": [{"role": "user", "content": question}]},
      stream_mode="values",
  ):
      if verbose:
        step["messages"][-1].pretty_print()
      else:
        if step['messages'][-1].response_metadata.get('finish_reason')=='stop':
          try:
            return display(Markdown(step['messages'][-1].content))
          except:
            return display(Markdown("I don't know"))

get_sql_reponse("Which is most profitable SKU_ID?")

The most profitable SKU_ID is 8, which corresponds to the product "t-shirt" in the category "women," with a profit of ₹34,128.

In [ ]:
get_sql_reponse("Which is most profitable SKU_ID?",verbose=True)

================================ Human Message =================================

Which is most profitable SKU_ID?
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_schema (call_mmY27W49Z3vPzzEI7Vt2kUNb)
 Call ID: call_mmY27W49Z3vPzzEI7Vt2kUNb
  Args:
    table_names: historical_data
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE historical_data (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	date TIMESTAMP, 
	unit_price REAL, 
	unit_cost REAL, 
	discount_pct REAL, 
	seasonality_factor REAL, 
	units_sold REAL, 
	revenue REAL, 
	profit REAL
)

/*
3 rows from historical_data table:
sku_id	product_name	category	date	unit_price	unit_cost	discount_pct	seasonality_factor	units_sold	revenue	profit
1	t-shirt	men	2023-06-01 00:00:00	1165.0	753.0	20.0	1.1	57.0	66405.0	23484.0
1	t-shirt	men	2023-07-01 00:00:00	1171.0	753.0	20.0	1.1	56.0	65576.

In [ ]:
db.run('select sku_id, product_name,	category, sum(profit) profit from historical_data group by sku_id, product_name,	category;')

"[(1, 't-shirt', 'men', 656949.0), (8, 't-shirt', 'women', 693898.0)]"

In [ ]:
get_sql_reponse("Which is most profitable SKU?")

The most profitable SKU is:
- **Product Name:** t-shirt
- **Category:** women
- **Total Profit:** ₹693,898.00

In [ ]:
get_sql_reponse("Which is most profitable SKU of 2025?",verbose=True)

================================ Human Message =================================

Which is most profitable SKU of 2025?
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_list_tables (call_EEypiOO91IQr5oIyF2ZoHarc)
 Call ID: call_EEypiOO91IQr5oIyF2ZoHarc
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

competitior_information, current_product_information, forecast_data, historical_data, inventory_data
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_schema (call_YtAUP4zzcugLmzZ835O6wZEN)
 Call ID: call_YtAUP4zzcugLmzZ835O6wZEN
  Args:
    table_names: historical_data
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE historical_data (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	date TIMESTAMP, 


In [ ]:
db.run('select sku_id, product_name,	category, sum(profit) profit from historical_data where date >= "2025-01-01" group by sku_id, product_name,	category;')

"[(1, 't-shirt', 'men', 188747.0), (8, 't-shirt', 'women', 196767.0)]"

In [ ]:
get_sql_reponse("Which is total profit of each SKU in year 2025?", verbose=True)

================================ Human Message =================================

Which is total profit of each SKU in year 2025?
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_list_tables (call_FggjzY05MfjeTUtYUOP2NeTl)
 Call ID: call_FggjzY05MfjeTUtYUOP2NeTl
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

competitior_information, current_product_information, forecast_data, historical_data, inventory_data
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_schema (call_O40ox5wDL0zb8YgbKzNGMgkP)
 Call ID: call_O40ox5wDL0zb8YgbKzNGMgkP
  Args:
    table_names: historical_data
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE historical_data (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	date TI

In [ ]:
get_sql_reponse("What is most elastic SKU?",verbose=True)

================================ Human Message =================================

What is most elastic SKU?
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_list_tables (call_yp8tHt1qSnhhO9CtKa1Ydtae)
 Call ID: call_yp8tHt1qSnhhO9CtKa1Ydtae
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

competitior_information, current_product_information, forecast_data, historical_data, inventory_data
================================== Ai Message ==================================
Name: Retail_Data_Agent
Tool Calls:
  sql_db_schema (call_clCn7myEDSzTNx1aoA72Qhz2)
 Call ID: call_clCn7myEDSzTNx1aoA72Qhz2
  Args:
    table_names: current_product_information
  sql_db_schema (call_V22GsloBB2czwVgN7pPRBR7l)
 Call ID: call_V22GsloBB2czwVgN7pPRBR7l
  Args:
    table_names: historical_data
================================= Tool Message ================================

In [ ]:
get_sql_reponse("compare our SKU's prices with competitor's prices?")

Here are the comparisons of our SKU's prices with competitor's prices:

1. **Product**: T-shirt (Men)
   - **Our Price**: ₹1460.0
   - **Competitor's Price**: ₹864.5

2. **Product**: T-shirt (Women)
   - **Our Price**: ₹1820.0
   - **Competitor's Price**: ₹946.4

These examples show that our prices are significantly higher than those of the competitors for both products.

In [16]:
from langchain.tools import tool

@tool("get_sql_reponse", return_direct=True)
def get_sql_reponse(question, verbose=False):
  """
  Answer data questions based on the input data available.
  """
  for step in data_agent.stream(
      {"messages": [{"role": "user", "content": question}]},
      stream_mode="values",
  ):
      if verbose:
        step["messages"][-1].pretty_print()
      else:
        if step['messages'][-1].response_metadata.get('finish_reason')=='stop':
          try:
            return display(Markdown(step['messages'][-1].content))
          except:
            return display(Markdown("I don't know"))

get_sql_reponse.invoke("Which is most profitable SKU_ID?")

The most profitable SKU_ID is **8**, which corresponds to the product **t-shirt** in the **women** category, with a total profit of **693,898 INR**.

In [21]:
llm.invoke([{"role": "user", "content":
                              """You are a great poet and philosopher in hindi language. Your task is to explain given poem line by line, what does it actually mean, in vey simple language. पाट ना पाया मीठा पानी ओर-छोर की दूरी रे मन कस्तूरी रे, जग दस्तूरी रे मन कस्तूरी रे"""}]).content

'यह कविता की एक पंक्ति का विश्लेषण करना बहुत रोचक है। चलिए, हम इसे पंक्ति दर पंक्ति समझते हैं:\n\n1. **"पाट ना पाया मीठा पानी"**  \n   इसका मतलब है कि मीठा पानी (सुख, खुशी या प्रेम) हमें नहीं मिला। "पाट" का अर्थ है कोई वस्तु जो बाधित कर रही है, यानी कुछ वजह है जिससे हमें खुशी नहीं मिल पा रही है।\n\n2. **" ओर-छोर की दूरी"**  \n   यहाँ "ओर-छोर" का मतलब है किनारे या सीमा। यह पंक्ति यह बताती है कि हम एक ऐसी स्थिति में हैं जहां सुख और दुख के बीच की दूरी बहुत ज्यादा है। इसका आर्थ है कि हमें अपने लक्ष्य तक पहुँचने के लिए बहुत लंबा रास्ता तय करना है।\n\n3. **"रे मन कस्तूरी रे"**  \n   इस पंक्ति में "मन" से तात्पर्य है दिल या आत्मा। "कस्तूरी" एक सुगंधित वस्तु है, जो कि प्रतीकात्मक रूप से हमारे अंदर की गहराइयों, हमारी इच्छाओं और भावनाओं का प्रतिनिधित्व करती है। यह पंक्ति दिल की गहराइयों से जुड़ी भावनाओं को दर्शाती है।\n\n4. **"जग दस्तूरी रे मन कस्तूरी रे"**  \n   "जग दस्तूरी" का अर्थ है जीवन की व्यावहारिकता या दुनिया की बनावट। यहाँ यह बताया जा रहा है कि हमारी अंदर की इच्छाएँ और गहरी भावनाएँ इस द

In [36]:
from datetime import date

today = date.today()


2025-10-18


In [37]:
from langchain.tools import tool
from datetime import date

@tool("get_today_date", return_direct=False)
def get_today_date():
    """
    Get today's date in YYYY-MM-DD format.
    """
    today = date.today()
    formatted = today.strftime("%Y-%m-%d")
    return formatted

@tool("simulate_demand_tool", return_direct=False)
def simulate_demand_tool(elasticity, discount = 0.1, P1=100, Q1=1000):
    """
    Simulate new demand and revenue given price change and elasticity.

    Parameters:
        elasticity (float): Price elasticity of demand (negative)
        discount (float): Discount (in range 0-1)
        P1 (float): Initial price
        Q1 (float): Initial quantity

    Returns:
        dict with new quantity, new revenue, % change in Q, % change in revenue
    """
    P2 = P1 * (1+discount)
    # % change in quantity
    pct_change_Q = elasticity * discount

    # new quantity
    Q2 = Q1 * (1 + pct_change_Q)

    # revenues
    R1 = P1 * Q1
    R2 = P2 * Q2

    return {
        "Initial Price": P1,
        "New Price": P2,
        "Initial Quantity": Q1,
        "New Quantity": round(Q2, 2),
        "Initial Revenue": round(R1, 2),
        "New Revenue": round(R2, 2),
        "% Change Quantity": round(pct_change_Q * 100, 2),
        "% Change Revenue": round(((R2 - R1) / R1) * 100, 2)
    }

# # Example runs:
# print(simulate_demand(-0.6, -0.1))
# print(simulate_demand(-1.5, -0.1))

# print(simulate_demand(-0.6, 0.1))
# print(simulate_demand(-1.5, 0.1))

## Connect data agent with Main Supervisor Agent

In [ ]:
!pip install -q langgraph-supervisor

In [ ]:
from langgraph_supervisor import create_supervisor

In [22]:
# from langgraph_supervisor import create_supervisor
# from langchain.chat_models import init_chat_model

# tools = [simulate_demand_tool, get_sql_reponse]

# supervisor = create_supervisor(
#     name="Main_Retail_Agent",
#     model=llm,
#     tools=tools,
#     agents=[data_agent],
#     prompt=(
#         "You are a supervisor agent responsible to build pricing strategy for a retail company:\n"
#         "When you get a question, plan it in detail, check all the tools and agents you can use. then execute step by step.\n"
#         "Use given data agent to assign any data related question to this agent and get answers\n"
#         "Use simulate_demand_tool to check impact of a price change in any SKU. Do not do price change calculation by your own.\n"
#         "Assign work to agents and build strategy using the data it returns.\n"
#         "All the price data is in INR."
#     ),
#     add_handoff_back_messages=True,
#     output_mode="full_history",
# ).compile()


In [62]:
# from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model

tools = [simulate_demand_tool, get_sql_reponse, get_today_date]
# from langchain.memory import ConversationBufferMemory
# memory = ConversationBufferMemory(
#     memory_key="chat_history",  # name used in prompt template
#     return_messages=True
# )
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

supervisor = create_react_agent(
    name="Main_Retail_Agent",
    model=llm,
    tools=tools,
    checkpointer=memory,
    prompt=(
        "You are a supervisor agent responsible to build pricing strategy for a retail company:\n"
        "When you get a question, plan it in detail, check all the tools and agents you can use. then execute step by step.\n"
        "Use given tools to get any support you need in handling respective tasks."
        "Use get_today_date to check today's date if needed\n"
        "Use get_sql_reponse to answer data related question\n"
        "Use simulate_demand_tool to check impact of a price change in any SKU. Do not do price change calculation by your own.\n"
        "Transfer tasks to tools and get your desired output and build strategy using the data it returns.\n"
        "All the price data is in INR."
    ),
)


/tmp/ipython-input-3464996402.py:13: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  supervisor = create_react_agent(


In [65]:
thread = {"configurable": {"thread_id": "session_1"}}
def chat_with_supervisor(input):
  for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": input,
            }
        ],
    },
    config=thread
  ):
    print([v['messages'] for k, v in chunk.items()])
    # print("Detail-->")
    # print([v['messages'][-1].content for k, v in chunk.items()])


chat_with_supervisor("Which is our highest revenue generating item?")

[[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IYhCGgPhjGbWresvUvWYPiKM', 'function': {'arguments': '{"question":"SELECT item_name FROM sales_data ORDER BY revenue DESC LIMIT 1;","verbose":true}', 'name': 'get_sql_reponse'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 343, 'total_tokens': 376, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CRswIIIXmt0A6wFFZtLrD82JXK0os', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='Main_Retail_Agent', id='run--ae916ccc-a606-40b8-935b-e201fe78615f-0', tool_calls=[{'name': 'get_sql_reponse', 'args': {'question': 'SELECT item_name FROM sales_data ORDER BY revenue DESC LIMIT

In [66]:
chat_with_supervisor("What is today's date?")

[[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_P2edY3Ybb2Fuw3YV1VlsaYF0', 'function': {'arguments': '{}', 'name': 'get_today_date'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 402, 'total_tokens': 413, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CRswO6Ih78VvKjKdCZWVDVarw06di', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='Main_Retail_Agent', id='run--181d5963-7cb8-46a4-80bb-38ff2e9cef11-0', tool_calls=[{'name': 'get_today_date', 'args': {}, 'id': 'call_P2edY3Ybb2Fuw3YV1VlsaYF0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 402, 'output_tokens': 11, 'total_tokens': 413, 'input_token_details':

In [69]:
chat_with_supervisor("What is today's date?")

[[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_X1xRVVUWW6i4rDl4zFbHhFg3', 'function': {'arguments': '{}', 'name': 'get_today_date'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 717, 'total_tokens': 728, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CRsy9cPitJDsLtn7uyWgcYVMyZ9la', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='Main_Retail_Agent', id='run--b9aff008-4a7b-4878-a3df-680b1fe24073-0', tool_calls=[{'name': 'get_today_date', 'args': {}, 'id': 'call_X1xRVVUWW6i4rDl4zFbHhFg3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 717, 'output_tokens': 11, 'total_tokens': 728, 'input_token_details':

In [72]:
chat_with_supervisor("What is expected sale this item in next 3 months?")

[[AIMessage(content='I am currently unable to retrieve the necessary data due to persistent issues with the database access. However, to estimate the expected sales of the highest revenue-generating item over the next 3 months, we would typically require:\n\n1. **Current Price of the Item**: The selling price at which the item is currently being sold.\n2. **Current Sales Volume**: The number of units currently sold, usually averaged over a recent time period.\n3. **Price Elasticity of Demand**: This indicates how sensitive the demand for the item is to changes in price.\n\nWithout this data, I cannot perform the expected sales simulation at this moment.\n\nIf you have access to this information or can provide the details, please share it with me so that I can proceed with the demand simulation for the next 3 months. Alternatively, if you wish to explore another aspect of pricing strategy, let me know!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion

In [ ]:
# chat_with_supervisor("What will happen if we increase our sku_id 1's price by 10%?")

In [ ]:
# chat_with_supervisor("What will happen if we decrease our sku_id 1's price by 10%? Describe in detail.")

In [73]:
chat_with_supervisor("I am considering a 10% price drop promotion in the store. What will be the impact on sales,revenue and profit? Do I have enough inventory for this? Describe in detail.")

[[AIMessage(content="To assess the impact of a 10% price drop promotion on sales, revenue, and profit, as well as to check if there is enough inventory for this, we will take the following systematic approach:\n\n### Steps to Follow:\n\n1. **Retrieve Current Data**:\n   - Get the current sales volume, price, and inventory levels for the highest revenue-generating item.\n   - Find out the cost of the item to assess profit margins.\n\n2. **Simulate Demand with Price Drop**:\n   - Use the current price and quantity sold, as well as the price elasticity of demand (which can be retrieved or estimated), to simulate the expected demand after a 10% price reduction.\n   - Calculate the new revenue and the overall effect on profit.\n\n3. **Assess Inventory Levels**:\n   - Check if the current inventory can meet the expected increase in demand due to the promotion.\n\n### Next Steps:\n\nSince I am currently unable to retrieve specific data regarding the sales volume, price, or inventory of the hi

In [74]:
chat_with_supervisor("Let's say I filled the stock. Then what is the impact on profit?")

[[AIMessage(content="Since we are operating under the assumption that the stock has been filled and we have the following parameters for the highest revenue-generating item (even though I could not retrieve them from the database):\n\n1. **Current Price (P1)**: Assume it's ₹100 (you can adjust this)\n2. **Current Quantity Sold (Q1)**: Assume it's 1000 units (you can adjust this)\n3. **Price Elasticity of Demand (elasticity)**: Let's assume it is -1.5 (you can adjust this)\n4. **Cost Price**: Assume ₹60 (you can adjust this)\n\n### Steps for Simulation:\n\n1. **Determine New Price**: \n   - New Price (P2) = Current Price - 10% of Current Price = ₹100 - (0.10 * ₹100) = ₹90\n\n2. **Simulate Demand**:\n   - Use the `simulate_demand_tool` to calculate new demand, revenue, and profit with the provided elasticity and price changes.\n\n3. **Calculate Revenue**:\n   - Use the new demand to estimate expected revenue (New Quantity * New Price).\n\n4. **Calculate Costs**:\n   - Use the new quantit

# We need to add memory and improve the quality of prompt to understand questions better

In [ ]:
def chat_with_supervisor(input):
  output = []
  # input = "Which is our highest revenue generating item?"
  # input = "How many SKU-1 are left in the inventory?"
  # input = "What if we increase price of SKU-1 by 10% ?"

  for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": input,
            }
        ]
    },
  ):
    for k,v in chunk.items():
      print(k)
      for v1 in chunk[k]['messages']:
        print(v1.content)
        output.append((k, v1.content))
  return output


In [ ]:
chat_with_supervisor("How many SKU-1 are sold last month?")

supervisor
How many SKU-1 are sold last month?

Successfully transferred to Retail_Data_Agent
Retail_Data_Agent
How many SKU-1 are sold last month?

Successfully transferred to Retail_Data_Agent

competitior_information, current_product_information, forecast_data, historical_data, inventory_data


CREATE TABLE historical_data (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	date TIMESTAMP, 
	unit_price REAL, 
	unit_cost REAL, 
	discount_pct REAL, 
	seasonality_factor REAL, 
	units_sold REAL, 
	revenue REAL, 
	profit REAL
)

/*
3 rows from historical_data table:
sku_id	product_name	category	date	unit_price	unit_cost	discount_pct	seasonality_factor	units_sold	revenue	profit
1	t-shirt	men	2023-06-01 00:00:00	1165.0	753.0	20.0	1.1	57.0	66405.0	23484.0
1	t-shirt	men	2023-07-01 00:00:00	1171.0	753.0	20.0	1.1	56.0	65576.0	23408.0
1	t-shirt	men	2023-08-01 00:00:00	1456.0	753.0	0.0	1.1	38.0	55328.0	26714.0
*/

CREATE TABLE current_product_information (
	sku_id INTEGER, 
	base_price RE

[('supervisor', 'How many SKU-1 are sold last month?'),
 ('supervisor', ''),
 ('supervisor', 'Successfully transferred to Retail_Data_Agent'),
 ('Retail_Data_Agent', 'How many SKU-1 are sold last month?'),
 ('Retail_Data_Agent', ''),
 ('Retail_Data_Agent', 'Successfully transferred to Retail_Data_Agent'),
 ('Retail_Data_Agent', ''),
 ('Retail_Data_Agent',
  'competitior_information, current_product_information, forecast_data, historical_data, inventory_data'),
 ('Retail_Data_Agent', ''),
 ('Retail_Data_Agent',
  '\nCREATE TABLE historical_data (\n\tsku_id INTEGER, \n\tproduct_name TEXT, \n\tcategory TEXT, \n\tdate TIMESTAMP, \n\tunit_price REAL, \n\tunit_cost REAL, \n\tdiscount_pct REAL, \n\tseasonality_factor REAL, \n\tunits_sold REAL, \n\trevenue REAL, \n\tprofit REAL\n)\n\n/*\n3 rows from historical_data table:\nsku_id\tproduct_name\tcategory\tdate\tunit_price\tunit_cost\tdiscount_pct\tseasonality_factor\tunits_sold\trevenue\tprofit\n1\tt-shirt\tmen\t2023-06-01 00:00:00\t1165.0\t753

In [ ]:
output = []
# input = "Which is our highest revenue generating item?"
# input = "How many SKU-1 are left in the inventory?"
input = "What if we increase price of SKU-1 by 10% ?"

for chunk in supervisor.stream(
  {
      "messages": [
          {
              "role": "user",
              "content": input,
          }
      ]
  },
):
  for k,v in chunk.items():
    print(k)
    for v1 in chunk[k]['messages']:
      print(v1.content)


supervisor
What if we increase price of SKU-1 by 10% ?

Successfully transferred to Retail_Data_Agent
Retail_Data_Agent
What if we increase price of SKU-1 by 10% ?

Successfully transferred to Retail_Data_Agent

competitior_information, current_product_information, forecast_data, historical_data, inventory_data


CREATE TABLE current_product_information (
	sku_id INTEGER, 
	base_price REAL, 
	base_demand REAL, 
	elasticity REAL, 
	margin REAL
)

/*
3 rows from current_product_information table:
sku_id	base_price	base_demand	elasticity	margin
1	1460.0	500.0	-1.06	0.94
8	1820.0	800.0	-0.93	1.08
*/


CREATE TABLE forecast_data (
	sku_id INTEGER, 
	product_name TEXT, 
	category TEXT, 
	date TIMESTAMP, 
	units_sale REAL
)

/*
3 rows from forecast_data table:
sku_id	product_name	category	date	units_sale
1	t-shirt	men	2025-09-01 00:00:00	41.0
1	t-shirt	men	2025-10-01 00:00:00	41.0
1	t-shirt	men	2025-11-01 00:00:00	41.0
*/


CREATE TABLE historical_data (
	sku_id INTEGER, 
	product_name TEXT, 

Data agent is doing most of the work, need to teach main agent to pass only relevant questions to data agent and do rest of the work